This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Apply calibration solutions to dataset and check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with only gain corruptions and a gain-only solver,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-04 10:56:22,828 - pipeline.bandpass_calibration - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-04 10:56:22,842 - workflow.utils - INFO - Using AA2-Low-ECP-240228 with 68 stations
2024-12-04 10:56:24,059 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-12-04 10:56:24,060 - workflow.utils - INFO - Using 37 components from /data/EOS_1/mit183/gleamegc.dat


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-04 10:56:28,352 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-04 10:56:29,053 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-04 10:56:29,054 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-04 10:56:29,900 - pipeline.bandpass_calibration - INFO - Setting input from demo.ms in 16-channel chunks
2024-12-04 10:56:30,065 - pipeline.bandpass_calibration - INFO - Generating /data/EOS_1/mit183/gleamegc.dat LSM < 5.0 deg > 1 Jy
2024-12-04 10:56:30,741 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-12-04 10:56:30,743 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12-04 10:56:30,770 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2024-12-04 10:56:30,823 - pipeline.bandpass_calibration - INFO - Running graph and returning calibration solutions
<frozen abc>:106: FutureWar

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-04 10:56:35,392 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-04 10:56:35,403 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-04 10:56:35,408 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 13.0 seconds


In [4]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-04 10:56:35,826 - pipeline.bandpass_calibration - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-04 10:56:35,841 - workflow.utils - INFO - Using AA2-Low-ECP-240228 with 68 stations
2024-12-04 10:56:36,470 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-12-04 10:56:36,471 - workflow.utils - INFO - Using 37 components from /data/EOS_1/mit183/gleamegc.dat


Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-04 10:56:40,662 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-04 10:56:40,664 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2024-12-04 10:56:41,285 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-04 10:56:41,286 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-04 10:56:42,091 - pipeline.bandpass_calibration - INFO - Setting input from demo.ms in 16-channel chunks
2024-12-04 10:56:42,193 - pipeline.bandpass_calibration - INFO - Generating /data/EOS_1/mit183/gleamegc.dat LSM < 5.0 deg > 1 Jy
2024-12-04 10:56:42,879 - processing_tasks.lsm_tmp - INFO - extracted 37 GLEAM components
2024-12-04 10:56:42,880 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12-04 10:56:42,907 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2024-12-04 10:56:42,965 - pipeline.bandpa

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2024-12-04 10:56:56,874 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2024-12-04 10:56:56,884 - pipeline.bandpass_calibration - INFO - Checking results
2024-12-04 10:56:56,886 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 21.5 seconds
